# Global Macro Training (100M Steps)
This notebook runs the trading agent training on 8 uncorrelated assets.

In [ ]:
# 1. Install Dependencies
# protobuf==3.20.3 is critical to fix 'MessageFactory' error in Kaggle (TensorBoard mismatch)
!pip install stable-baselines3 shimmy>=0.2.1 protobuf==3.20.3 pandas_ta

In [ ]:
# 2. Copy Code to Working Directory
# Copying from 'global-macro-codes' dataset
!cp /kaggle/input/global-macro-codes/*.py ./

# 3. Overwrite Training Script with FIX (Column Capitalization & Date Indexing)
This cell overwrites `train_global_kaggle.py`. 
**Fixes:**
1.  **Capitalization**: Forces `close` -> `Close`.
2.  **Date Indexing**: Moves 'Date' column to the Index so it doesn't crash the math.

In [ ]:
%%writefile train_global_kaggle.py
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize
from stable_baselines3.common.callbacks import EvalCallback
import os

from stock_env import StockTradingEnv
from data_processor import add_technical_indicators, clean_data, load_data

def make_env(rank, seed=0):
    """
    Creates an environment for a specific asset based on the CPU rank.
    """
    def _init():
        # The "Global Macro" 8-Pack list
        assets = [
            'EUR_USD',      # 0
            'GBP_USD',      # 1
            'SPX500_USD',   # 2
            'JP225_USD',    # 3
            'XAU_USD',      # 4
            'WTICO_USD',    # 5
            'USB10Y_USD',   # 6
            'CORN_USD'      # 7
        ]
        
        # Select asset by rank
        asset_name = assets[rank % len(assets)]
        
        # Load merged data (created by prepare_global_data.py)
        # Kaggle Path (Dataset name: global-macro-data)
        file_path = f"/kaggle/input/global-macro-data/{asset_name}_merged.csv"
        
        print(f"[Rank {rank}] Loading {asset_name} from {file_path}...")
        
        if not os.path.exists(file_path):
             print(f"ERROR: File not found {file_path}. Is data prep done?")
             raise FileNotFoundError(f"{file_path}")
             
        df = pd.read_csv(file_path)
        
        # FIX 1: Renaming columns to Title Case
        df.columns = [c.capitalize() for c in df.columns]

        # FIX 2: Set Date as Index (Critical logic from data_processor.load_data)
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
        
        # Preprocessing on the fly
        df = add_technical_indicators(df)
        df = clean_data(df)
        
        # Create environment with Random Start
        env = StockTradingEnv(
             df=df, 
             commission_pct=0.0001, 
             leverage=20.0, # Futures Leverage 1:20 (User Requested)
             random_start=True,
             stop_loss_pct=0.01, # 1% Risk
             take_profit_pct=0.03 # 3% Reward (1:3 Ratio)
        )
        env.reset(seed=seed + rank)
        return env
    return _init

if __name__ == "__main__":
    # Ensure output dirs
    os.makedirs("checkpoints/global_macro", exist_ok=True)
    os.makedirs("logs/global_macro", exist_ok=True)
    
    # Number of parallel environments = 8 assets
    num_cpu = 8 
    
    print(f"Starting Global Macro Training on {num_cpu} cores...")
    
    # Create the parallel environments
    # Normalizing Observations and Rewards (CRITICAL adaptation from past model)
    # This helps the model handle different price scales (e.g. Gold 2000 vs EUR 1.05)
    env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Setup Eval Callback to save best model during the long run
    eval_callback = EvalCallback(
        env, 
        best_model_save_path='./checkpoints/global_macro/best_model',
        log_path='./logs/global_macro',
        eval_freq=100_000,
        deterministic=True,
        render=False
    )

    # Define the model (PPO)
    # Refined Settings from past success:
    # - net_arch=[128, 128]: Deeper network for complex patterns
    # - batch_size=512: Better gradient estimation
    # - ent_coef=0.01: High exploration for multi-asset
    policy_kwargs = dict(net_arch=dict(pi=[128, 128], vf=[128, 128]))
    
    model = PPO(
        "MlpPolicy", 
        env, 
        verbose=1, 
        ent_coef=0.01, 
        learning_rate=0.0003,
        batch_size=512,
        policy_kwargs=policy_kwargs,
        tensorboard_log="./tensorboard_logs_global"
    )
    
    # Train
    TOTAL_TIMESTEPS = 100_000_000
    print(f"Starting training for {TOTAL_TIMESTEPS} timesteps...")
    print("Strategy: Global Macro (8 Assets) | Random Start | Normalized | Recurrent-Ready")
    
    try:
        model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=eval_callback)
        model.save("checkpoints/global_macro/ppo_global_macro_final")
        env.save("checkpoints/global_macro/vec_normalize.pkl") # Save normalization stats!
        print("Training Complete. Model and Normalization stats saved.")
    except Exception as e:
        print(f"Training interrupted or failed: {e}")
        model.save("checkpoints/global_macro/ppo_global_macro_interrupted")
        env.save("checkpoints/global_macro/vec_normalize_interrupted.pkl")

In [ ]:
# 4. Run Training
!python train_global_kaggle.py